<a href="https://colab.research.google.com/github/hanseul1215/ESAA_study/blob/master/0328_ensemble_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [ ]:
X,y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [5]:
for estimator in [rf_clf, et_clf, svm_clf, mlp_clf]:
  estimator.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [7]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('svm', svm_clf), ('mlp', mlp_clf)]
)
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [8]:
from sklearn.metrics import accuracy_score
pred=[]
for estimator in [rf_clf, et_clf, svm_clf, mlp_clf]:
  pred.append(estimator.predict)

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [9]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('mlp', mlp_clf)]
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [10]:
voting_clf.score(X_test, y_test)

0.9741

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [11]:
voting_clf.voting = "soft"
voting_clf.score(X_test, y_test
                 )

0.9722